In [1]:
import numpy as np
import pandas as pd
import os
os.chdir('/projects/sccn/weili_A1/liacz/Documents/Bilingualism_CVAE')

## load brain data

In [3]:
indir = '/mmfs1/data/liacz/Documents/Bilingualism_CVAE/data/array_brains'
# indir = '/mmfs1/data/liacz/Documents/Bilingualism_CVAE/data/array_brains/unaligned'

BRAIN_data = np.empty([0,64,64,64])
BRAIN_subs = np.empty([0,])
BRAIN_ds = np.empty([0,])
for dat in os.listdir(indir): 
    if dat.startswith('Anat-Bilingual-64iso'):
#     if dat.startswith('Anat-Bilingual-64iso-H'):
        arr = np.load(os.path.join(indir,dat))
        BRAIN_data = np.concatenate((BRAIN_data, arr['data']), axis=0)
        BRAIN_subs = np.concatenate((BRAIN_subs, arr['subs']), axis=0)
        BRAIN_ds = np.concatenate((BRAIN_ds, [dat.split('.')[0].split('-')[-2]]*len(arr['subs'])), axis=0)

In [4]:
BRAIN_data.shape

(2499, 64, 64, 64)

## load demographic information

In [4]:
indir = 'data/legend'
outdir = 'data/legend.csv'
fns = [fn for fn in os.listdir(indir) if fn.endswith('.csv')]
legend = pd.DataFrame()
dataframes = []  # Create a list to store dataframes

for fn in fns:
    df_temp = pd.read_csv(os.path.join(indir,fn))
    print(len(df_temp))
    dataframes.append(df_temp)  # Add to list instead of using append

# Use pd.concat to combine all dataframes at once
legend = pd.concat(dataframes, ignore_index=True)

# legend.loc[legend['ds']=='HCPaging', 'participant_id'] = legend.loc[legend['ds']=='HCPaging', 'participant_id']+'_V1_MR'
# legend.loc[legend['ds']=='HCPdev', 'participant_id'] = legend.loc[legend['ds']=='HCPdev', 'participant_id']+'_V1_MR'
# legend.loc[legend['ds']=='ping', 'ds'] = 'Ping'
# legend.loc[legend['ds']=='ping', 'ds'] = 'ping'

legend.loc[legend['ds']=='Hernandez', 'participant_id'] = legend.loc[legend['ds']=='Hernandez', 'participant_id']+'.mgz'

legend['ds_par_id'] = legend['ds'] +'_'+ legend['participant_id']
legend.to_csv(outdir, index=False)

92
363


In [5]:
legend

,participant_id,bilingualism,age,sex,aoa_group,num_lan,sec_lan_proficiency,ds,scanner,aoa,fst_lan_proficiency,ds_par_id
0,sub-2975,0.0,21.0,F,NaN,3.0,NaN,ds1747,Erlangen,NaN,NaN,ds1747_sub-2975
1,sub-3156,1.0,22.0,M,LB,4.0,1.000000,ds1747,Erlangen,NaN,NaN,ds1747_sub-3156
2,sub-3225,1.0,25.0,M,LB,2.0,0.800000,ds1747,Erlangen,NaN,NaN,ds1747_sub-3225
3,sub-3788,0.0,21.0,F,NaN,NaN,NaN,ds1747,Erlangen,NaN,NaN,ds1747_sub-3788
4,sub-3789,0.0,25.0,M,NaN,NaN,NaN,ds1747,Erlangen,NaN,NaN,ds1747_sub-3789
...,...,...,...,...,...,...,...,...,...,...,...,...
450,UH_T.mgz,0.0,19.0,F,NaN,NaN,0.790447,Hernandez,Hernandez,NaN,NaN,Hernandez_UH_T.mgz
451,UH_U.mgz,0.0,21.0,M,NaN,NaN,0.859527,Hernandez,Hernandez,NaN,NaN,Hernandez_UH_U.mgz
452,UH_V.mgz,1.0,18.0,M,EB,NaN,0.702619,Hernandez,Hernandez,4.0,0.532866,Hernandez_UH_V.mgz
453,UH_Y.mgz,1.0,22.0,F,LB,NaN,0.717750,Hernandez,Hernandez,10.0,0.752700,Hernandez_UH_Y.mgz


## align two datasets

In [ ]:
BRAIN_ds_subs = [BRAIN_ds[i]+'_'+BRAIN_subs[i] for i in range(0,len(BRAIN_subs))]
legend_subs = legend.ds_par_id

In [8]:
print(len(legend_subs))
print(legend_subs[3300])
print(legend_subs[1800])
print(legend_subs[900])

3324
Ping_sub-P0358
HCPaging_HCA9640889_V1_MR
Hernandez_WL_025.mgz


In [9]:
BRAIN_ds_subs[700]

'Ping_sub-P1693'

In [10]:
# BRAIN_ds_subs

In [11]:
subs_legend = [sub in BRAIN_ds_subs for sub in legend_subs]
subs_BRAIN = [sub in list(legend_subs) for sub in BRAIN_ds_subs]

In [12]:
print(len(subs_legend))
print(len(subs_BRAIN))

3324
2499


In [13]:
BRAIN_data = BRAIN_data[subs_BRAIN]
BRAIN_subs = BRAIN_subs[subs_BRAIN]
BRAIN_ds = BRAIN_ds[subs_BRAIN]

In [14]:
legend = legend[subs_legend]

In [15]:
print(len(BRAIN_data))
print(len(legend))

2497
2497


## save data

In [16]:
outdir = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/array_brains')
np.save(os.path.join(outdir, ("arr_combine/BRAIN_data")), BRAIN_data)
np.save(os.path.join(outdir, ("arr_combine/BRAIN_subs")), BRAIN_subs)
np.save(os.path.join(outdir, ("arr_combine/BRAIN_ds")), BRAIN_ds)

In [17]:
len(legend)

2497

In [18]:
legend.to_csv(
    os.path.expanduser('~/Documents/Bilingualism_CVAE/data/legend.csv'),
    index = False)

## create bilingual and monolingual data

In [19]:
#BRAIN_data = np.array(BRAIN_data)
nsubs = BRAIN_data.shape[0]
print([arr.shape for arr in [BRAIN_ds, BRAIN_subs, BRAIN_data]])
print(nsubs)
print((BRAIN_data.min(),BRAIN_data.max()))

[(2497,), (2497,), (2497, 64, 64, 64)]
2497
(0.0, 1.0)


In [20]:
df = pd.read_csv('/mmfs1/data/liacz/Documents/Bilingualism_CVAE/data/legend.csv')
BI_subs = df.loc[df['bilingualism'].values == 1.0, 'ds_par_id']
MO_subs = df.loc[df['bilingualism'].values == 0.0, 'ds_par_id']
BRAIN_ds_subs = [BRAIN_ds[i]+'_'+BRAIN_subs[i] for i in range(0,len(BRAIN_subs))]
BI_subs = [sub in BI_subs.to_list() for sub in BRAIN_ds_subs]
MO_subs = [sub in MO_subs.to_list() for sub in BRAIN_ds_subs]

In [21]:
print(BI_subs.count(True))
print(MO_subs.count(True))

853
1644


In [22]:
TD_subs = BRAIN_data[MO_subs,:,:,:] # Data of Monolinguals 
print(TD_subs.shape)

(1644, 64, 64, 64)


In [23]:
DX_subs = BRAIN_data[BI_subs,:,:,:] # Data of Bilinguals
print(DX_subs.shape)

(853, 64, 64, 64)


In [24]:
outdir = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/array_brains')
np.save(os.path.join(outdir, ("arr_combine/TD_subs")), TD_subs)
np.save(os.path.join(outdir, ("arr_combine/DX_subs")), DX_subs)